# ios上架有一堆解析度需要調整，這裡做個工具幫忙調整

## MAC FFMPEG 安裝方式
```
brew install ffmpeg
```
https://juejin.cn/post/6844904191433900040
https://stackoverflow.com/questions/54161418/python3-6-add-audio-to-cv2-processed-video

## 函式庫先執行

In [1]:
#opencv內建的Resize 比例會跑掉，這是用裁切的方式等比例放大或是縮小
import cv2
def AkingResize(img , OutPutWidth , OutPutHeight):
    InimgW = img.shape[1]
    InimgH = img.shape[0]
    InRatio = InimgW / InimgH
    OutRatio = OutPutWidth / OutPutHeight
    if (OutRatio > InRatio): #更扁平 所以縮放的時候寬度一樣，原圖的高度要裁切
        #print ("壓扁")
        NewHeight = int(InimgW * OutPutHeight / OutPutWidth)
        CenterY = InimgH / 2
        StartY = int(CenterY - NewHeight / 2)
        EndY = StartY + NewHeight
        NewImg = img[StartY:EndY,0:InimgW]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    elif (OutRatio < InRatio): #更高長 所以縮放的時候高度一樣，原圖的寬度要裁切
        #print ("長高")
        NewWidth = int(InimgH * OutPutWidth / OutPutHeight)
        CenterX = InimgW / 2
        StartX = int(CenterX - NewWidth / 2)
        EndX = StartX + NewWidth
        NewImg = img[0:InimgH,StartX:EndX]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    else: #相同比例
        Resizedimage = cv2.resize(img, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    return Resizedimage

def ResizePic(InputPicPathName , OutPutWidth,OutPutHeight,OutputPicPathName):
    #=============開始轉換====================================
    img = cv2.imread(InputPicPathName)
    if (img.any() != None):
        Resizedimage = AkingResize(img , OutPutWidth , OutPutHeight)
        cv2.imwrite(OutputPicPathName, Resizedimage)
        # 顯示圖片
        #cv2.imshow('Resized image', Resizedimage)
        #cv2.waitKey(3)
        #cv2.destroyAllWindows()
        print ("圖片轉換完成")
        # 按下任意鍵則關閉所有視窗
    else:
        print ("圖片讀取失敗")
        
        

## 照片調整

#### 設定sourcePath 的資料夾，把整個資料夾內的所有圖片轉換解析度  
2688 6.5寸  
2208 5.5寸  
2732 12.9寸

In [2]:
from os import listdir
from os.path import isfile, isdir, join
import os
sourcePath = r'/Users/aking/Desktop/Temp/pic'


def DoPic(sourcePath ,OutPutWidth ,OutPutHeight):
    files = listdir(sourcePath)
    OutputPicPath = join(sourcePath ,"{}".format(OutPutHeight ))
    if not os.path.isdir(OutputPicPath):
        os.mkdir(OutputPicPath)
    for file in files:
        #先搞定資料夾
        print (file)
        if (file == ".DS_Store"):
            continue
        InputPicPathName = join(sourcePath ,file )
        if (os.path.isfile(InputPicPathName)):
            OutputPicPathName = join(OutputPicPath ,file )
            ResizePic(InputPicPathName , OutPutWidth,OutPutHeight,OutputPicPathName)
        
DoPic(sourcePath , 1242 ,2688 )   
DoPic(sourcePath , 1242 ,2208 ) 
DoPic(sourcePath , 2048 ,2732 ) 

#1242 2688. 
#1242 2208. 
#2048 2732

.DS_Store
4.png
圖片轉換完成
5.png
圖片轉換完成
6.png
圖片轉換完成
2.png
圖片轉換完成
3.png
圖片轉換完成
1.png
圖片轉換完成
2688
.DS_Store
4.png
圖片轉換完成
5.png
圖片轉換完成
6.png
圖片轉換完成
2.png
圖片轉換完成
3.png
圖片轉換完成
1.png
圖片轉換完成
2688
.DS_Store
4.png
圖片轉換完成
5.png
圖片轉換完成
6.png
圖片轉換完成
2.png
圖片轉換完成
3.png
圖片轉換完成
2208
1.png
圖片轉換完成


## 影片調整

In [3]:
#影片調整函式
import cv2
import os

def ResizeVideo(InputVideoPathName , OutPutWidth,OutPutHeight,OutputVideoPathName):
    #=======================輸入設定====================================
    video = cv2.VideoCapture(InputVideoPathName)
    # 取得影像的尺寸大小
    width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = video.get(cv2.CAP_PROP_FPS) #每秒鐘幾張
    print ("原始寬度{} 高度{} FPS{}".format(width , height,fps))


    #=======================輸出檔案設定=======================
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    VideoFilePath = r"{}/TmpVideo.mp4".format(os.path.dirname(InputVideoPathName)) #解析度調整後的畫面
    SoundFilePath = r"{}/sound.mp3".format(os.path.dirname(InputVideoPathName))    #原始檔案分離出聲音
    VideoWithSoundFilePath = r"{}/TmpVideoWithSound.mp4".format(os.path.dirname(InputVideoPathName)) #畫面與聲音合成影片
    #畫面與聲音合成影片 之後需要再轉檔一次 app store才會接受

    out = cv2.VideoWriter(VideoFilePath, fourcc, fps, (int(OutPutWidth),  int(OutPutHeight)))
    #=============開始轉換====================================
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        image = AkingResize(frame, OutPutWidth, OutPutHeight)
        out.write(image)
        cv2.imshow('frame', image)
        if cv2.waitKey(1) == ord('q'):
            break
    video.release()
    out.release()

    CMD = r"ffmpeg -i {} -f mp3 -ab 256000 -vn {}".format(InputVideoPathName,SoundFilePath) #原始檔案分離出聲音
    os.system(CMD) #原始檔案分離出聲音


    CMD1 = r"ffmpeg -i {} -i {} -c copy -map 0:v:0 -map 1:a:0 {}".format(VideoFilePath, SoundFilePath , VideoWithSoundFilePath)
    os.system(CMD1) #畫面與聲音合成影片

    CMD2 = "ffmpeg -i {} -filter_complex 'scale={}:{},framerate=30' {}".format(VideoWithSoundFilePath , OutPutWidth,OutPutHeight,OutputVideoPathName)
    os.system(CMD2) #最後再轉檔案一次 App store才會吃



    os.system("rm -f {}".format(VideoFilePath)) 
    os.system("rm -f {}".format(SoundFilePath)) 
    os.system("rm -f {}".format(VideoWithSoundFilePath)) 



    print("轉換完成")
    cv2.destroyAllWindows() #MAC無法關閉視窗 就算了
    

6.5 吋 886 x 1920 像素（直向）  
5.5 吋  1080 x 1920   
12.9 吋   1200 x 1600   

# 單一影片調整

In [4]:
InputVideoPathName = r'/Users/aking/Desktop/Temp/ShrinkTutorial.mov' #輸入影像的路徑與檔名
basename = os.path.basename(InputVideoPathName)
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/{}{}.mp4'.format(basename[:-4] , OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  

轉換完成


frame=  893 fps= 56 q=-1.0 Lsize=    7802kB time=00:00:29.80 bitrate=2144.5kbits/s speed=1.87x    
video:7760kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.429150%
[libx264 @ 0x7fcfb0708040] frame I:10    Avg QP:17.54  size: 60154
[libx264 @ 0x7fcfb0708040] frame P:238   Avg QP:21.58  size: 17363
[libx264 @ 0x7fcfb0708040] frame B:645   Avg QP:23.73  size:  4980
[libx264 @ 0x7fcfb0708040] consecutive B-frames:  2.5%  2.9%  2.4% 92.3%
[libx264 @ 0x7fcfb0708040] mb I  I16..4: 23.0% 66.9% 10.1%
[libx264 @ 0x7fcfb0708040] mb P  I16..4:  6.1% 11.5%  0.9%  P16..4: 28.6%  5.5%  3.2%  0.0%  0.0%    skip:44.1%
[libx264 @ 0x7fcfb0708040] mb B  I16..4:  0.7%  0.6%  0.0%  B16..8: 26.9%  1.5%  0.1%  direct: 2.1%  skip:68.1%  L0:43.6% L1:53.7% BI: 2.7%
[libx264 @ 0x7fcfb0708040] 8x8 transform intra:60.8% inter:87.7%
[libx264 @ 0x7fcfb0708040] coded y,uvDC,uvAC intra: 31.5% 29.5% 7.6% inter: 8.0% 10.2% 0.3%
[libx264 @ 0x7fcfb0708040] i16 v,h,dc,p: 34% 40% 13% 13%
[l

# 批次影片轉檔案

In [5]:
InputVideoPathName = r'/Users/aking/Desktop/Temp/ShrinkTutorial.mov' #輸入影像的路徑與檔名

basename = os.path.basename(InputVideoPathName)
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/{}{}.mp4'.format(basename[:-4],OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

OutPutWidth = 1080           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/{}{}.mp4'.format(basename[:-4],OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

OutPutWidth = 1200          #輸出寬度
OutPutHeight = 1600                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/{}{}.mp4'.format(basename[:-4],OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

print ("全部完成～～")

原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  

frame=  893 fps= 41 q=-1.0 Lsize=    7802kB time=00:00:29.80 bitrate=2144.5kbits/s speed=1.38x    
video:7760kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.429150%
[libx264 @ 0x7fb571708300] frame I:10    Avg QP:17.54  size: 60154
[libx264 @ 0x7fb571708300] frame P:238   Avg QP:21.58  size: 17363
[libx264 @ 0x7fb571708300] frame B:645   Avg QP:23.73  size:  4980
[libx264 @ 0x7fb571708300] consecutive B-frames:  2.5%  2.9%  2.4% 92.3%
[libx264 @ 0x7fb571708300] mb I  I16..4: 23.0% 66.9% 10.1%
[libx264 @ 0x7fb571708300] mb P  I16..4:  6.1% 11.5%  0.9%  P16..4: 28.6%  5.5%  3.2%  0.0%  0.0%    skip:44.1%
[libx264 @ 0x7fb571708300] mb B  I16..4:  0.7%  0.6%  0.0%  B16..8: 26.9%  1.5%  0.1%  direct: 2.1%  skip:68.1%  L0:43.6% L1:53.7% BI: 2.7%
[libx264 @ 0x7fb571708300] 8x8 transform intra:60.8% inter:87.7%
[libx264 @ 0x7fb571708300] coded y,uvDC,uvAC intra: 31.5% 29.5% 7.6% inter: 8.0% 10.2% 0.3%
[libx264 @ 0x7fb571708300] i16 v,h,dc,p: 34% 40% 13% 13%
[l

轉換完成
原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  

[Parsed_framerate_1 @ 0x7fc047c04900] time base:1/15360 -> 1/15360 exact:1
[Parsed_framerate_1 @ 0x7fc047c04900] fps -> fps:30/1 scene score:8.200000 interpolate start:15 end:240
[libx264 @ 0x7fc042206d00] using SAR=1/1
[libx264 @ 0x7fc042206d00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7fc042206d00] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7fc042206d00] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=12 lookahead_threads=2 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree

轉換完成
原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  

轉換完成
全部完成～～


frame=  893 fps= 39 q=-1.0 Lsize=    9388kB time=00:00:29.80 bitrate=2580.4kbits/s speed=1.29x    
video:9346kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.356306%
[libx264 @ 0x7fb35cf08980] frame I:11    Avg QP:17.52  size: 50269
[libx264 @ 0x7fb35cf08980] frame P:239   Avg QP:22.05  size: 20937
[libx264 @ 0x7fb35cf08980] frame B:643   Avg QP:24.33  size:  6241
[libx264 @ 0x7fb35cf08980] consecutive B-frames:  2.7%  2.9%  3.0% 91.4%
[libx264 @ 0x7fb35cf08980] mb I  I16..4: 21.8% 71.1%  7.1%
[libx264 @ 0x7fb35cf08980] mb P  I16..4:  7.0% 14.6%  0.9%  P16..4: 28.8%  6.1%  3.0%  0.0%  0.0%    skip:39.7%
[libx264 @ 0x7fb35cf08980] mb B  I16..4:  0.8%  0.8%  0.0%  B16..8: 26.9%  1.8%  0.2%  direct: 2.1%  skip:67.3%  L0:43.9% L1:53.4% BI: 2.7%
[libx264 @ 0x7fb35cf08980] 8x8 transform intra:63.6% inter:88.3%
[libx264 @ 0x7fb35cf08980] coded y,uvDC,uvAC intra: 31.8% 29.0% 6.6% inter: 9.1% 10.1% 0.3%
[libx264 @ 0x7fb35cf08980] i16 v,h,dc,p: 33% 39% 13% 15%
[l

# 以下懷舊程式碼無須理會

In [26]:
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "scale=1200:1600,framerate=30;"  /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout.mp4
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "[0:v]setpts=1*PTS,scale=1200:1600,framerate=30[v];[0:a]atempo=8.0[a]" -map "[v]" -map "[a]" /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout1600.mp4

ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Tmp.mp4 -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/sound.aac -c copy -map 0:v:0 -map 1:a:0 /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava886S.mp4


In [46]:
import cv2
import os

#InputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/RestoreAndPurchases.mov'  #輸入影像的路徑與檔名
#OutPutWidth = 560            #輸出寬度
#OutPutHeight = 1200                 #輸出高度
#OutputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/RestoreAndPurchases.mp4'   #輸出影像的路徑與檔名



InputVideoPathName = r'/Users/aking/Desktop/Temp/Purchase_RestorePurchases.mov' #輸入影像的路徑與檔名
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/Purchase_RestorePurchases{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名


#=======================輸入設定====================================
video = cv2.VideoCapture(InputVideoPathName)
# 取得影像的尺寸大小
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = video.get(cv2.CAP_PROP_FPS) #每秒鐘幾張
print ("原始寬度{} 高度{} FPS{}".format(width , height,fps))


#=======================輸出檔案設定=======================
fourcc = cv2.VideoWriter_fourcc(*'avc1')
VideoFilePath = r"{}/TmpVideo.mp4".format(os.path.dirname(InputVideoPathName)) #解析度調整後的畫面
SoundFilePath = r"{}/sound.mp3".format(os.path.dirname(InputVideoPathName))    #原始檔案分離出聲音
VideoWithSoundFilePath = r"{}/TmpVideoWithSound.mp4".format(os.path.dirname(InputVideoPathName)) #畫面與聲音合成影片
#畫面與聲音合成影片 之後需要再轉檔一次 app store才會接受

out = cv2.VideoWriter(VideoFilePath, fourcc, fps, (int(OutPutWidth),  int(OutPutHeight)))
#=============開始轉換====================================
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    image = AkingResize(frame, OutPutWidth, OutPutHeight)
    out.write(image)
    cv2.imshow('frame', image)
    if cv2.waitKey(1) == ord('q'):
        break
video.release()
out.release()

CMD = r"ffmpeg -i {} -f mp3 -ab 256000 -vn {}".format(InputVideoPathName,SoundFilePath) #原始檔案分離出聲音
os.system(CMD) #原始檔案分離出聲音


CMD1 = r"ffmpeg -i {} -i {} -c copy -map 0:v:0 -map 1:a:0 {}".format(VideoFilePath, SoundFilePath , VideoWithSoundFilePath)
os.system(CMD1) #畫面與聲音合成影片

CMD2 = "ffmpeg -i {} -filter_complex 'scale={}:{},framerate=30' {}".format(VideoWithSoundFilePath , OutPutWidth,OutPutHeight,OutputVideoPathName)
os.system(CMD2) #最後再轉檔案一次 App store才會吃



os.system("rm -f {}".format(VideoFilePath)) 
os.system("rm -f {}".format(SoundFilePath)) 
os.system("rm -f {}".format(VideoWithSoundFilePath)) 



print("轉換完成")
cv2.destroyAllWindows()

原始寬度1170.0 高度2532.0 FPS41.09424851746791


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

轉換完成


frame=  599 fps= 67 q=-1.0 Lsize=    3716kB time=00:00:20.13 bitrate=1511.4kbits/s speed=2.24x    
video:3687kB audio:6kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.611658%
[libx264 @ 0x7fe601205b80] frame I:7     Avg QP:17.46  size: 36329
[libx264 @ 0x7fe601205b80] frame P:181   Avg QP:18.80  size: 10009
[libx264 @ 0x7fe601205b80] frame B:411   Avg QP:18.42  size:  4159
[libx264 @ 0x7fe601205b80] consecutive B-frames:  6.3%  3.3%  9.5% 80.8%
[libx264 @ 0x7fe601205b80] mb I  I16..4: 43.5% 46.8%  9.7%
[libx264 @ 0x7fe601205b80] mb P  I16..4: 10.9% 14.3%  1.4%  P16..4: 11.5%  1.8%  0.8%  0.0%  0.0%    skip:59.3%
[libx264 @ 0x7fe601205b80] mb B  I16..4:  1.4%  1.3%  0.2%  B16..8: 13.7%  1.5%  0.2%  direct: 1.9%  skip:79.9%  L0:47.2% L1:49.9% BI: 2.9%
[libx264 @ 0x7fe601205b80] 8x8 transform intra:51.5% inter:86.3%
[libx264 @ 0x7fe601205b80] coded y,uvDC,uvAC intra: 20.2% 29.1% 5.1% inter: 3.6% 5.3% 0.1%
[libx264 @ 0x7fe601205b80] i16 v,h,dc,p: 57% 26%  6% 10%
[li